In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\Qiskit\VQE\OptimizeHyperparams\popsize"
potential = 'AHO'
cutoff = 8

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []
init = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    #tolerance.append(optimzer['tolerance'])
    #abs_tolerance.append(optimzer['abs_tolerance'])
    #strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])
    init.append(optimzer['init'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [3]:
df = pd.DataFrame({
    'popsize': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    #'tolerance': tolerance,
    #'abs_tolerance': abs_tolerance,
    #'strategy': strategy,
    #'popsize': popsize,
    'init': init,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [4]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,popsize,num_vqe,shots,init,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
1,15,100,10000,scaled_samples,0/40,10000,10000,6.77283e+00,4.90442e+00,3.20101e-02,"11 days, 17:38:01.508244"
0,10,100,10000,scaled_samples,0/40,10000,10000,6.03957e+00,4.68912e+00,3.20101e-02,"7 days, 7:59:19.142563"
4,30,100,10000,scaled_samples,0/40,10000,10000,6.02342e+00,5.02808e+00,3.20101e-02,"21 days, 11:50:54.212644"
3,25,100,10000,scaled_samples,0/40,10000,10000,5.97792e+00,4.99462e+00,3.20101e-02,"18 days, 2:11:08.465381"
2,20,100,10000,scaled_samples,0/40,10000,10000,5.75120e+00,4.98858e+00,3.20101e-02,"14 days, 15:29:35.014212"
5,5,100,10000,scaled_samples,0/40,10000,10000,5.07281e+00,3.76267e+00,3.20101e-02,"3 days, 16:02:23.274535"


In [5]:
from tabulate import tabulate
df['popsize'] = df['popsize'].astype(int)
df = df.drop(columns=['init','num_vqe','shots','maxiter']).sort_values('popsize',ascending=True)

latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{rlrrrrl}
\hline
   popsize & Converged Runs   &   Mean Iter &   Delta min E &   Delta median E &    Exact & Time           \\
\hline
         5 & 32/40            &        2652 &   0.203387    &      2.04944     & 0.891599 & 0:34:13.146936 \\
        10 & 38/40            &         933 &   7.31637e-14 &      0.0243449   & 0.891599 & 0:24:01.104046 \\
        15 & 40/40            &         316 &   7.31637e-14 &      0.00182009  & 0.891599 & 0:13:37.611176 \\
        20 & 40/40            &         143 &   7.31637e-14 &      4.15225e-05 & 0.891599 & 0:10:10.726670 \\
        25 & 40/40            &         183 &   7.31637e-14 &      4.15022e-05 & 0.891599 & 0:14:41.362001 \\
        30 & 40/40            &         141 &   7.31637e-14 &      4.15022e-05 & 0.891599 & 0:14:25.687482 \\
\hline
\end{tabular}
